In [1]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import numpy as np
import random
from colour import Color
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
K = 25

In [3]:

locs = pd.read_csv("Locations_O3_full.csv")
locs["stationid"] = locs["stationid"].astype("int64")
locs = locs.set_index("stationid")


In [4]:

tooltip = "Click me!"
print()
ramp = list(Color('blue').range_to(Color('red'), 1000))
#folium.Marker(
#    [35.00139, -78.99055], popup="<i>John Griffin Middle School, 5551, Fisher Road, Fayetteville, Cumberland County, North Carolina, 28304, United States</i>", tooltip=tooltip
#).add_to(m)

In [5]:
locs

,lat,lon,name,state,elevation,distance,rating
stationid,,,,,,,
370510010,35.00139,-78.99055,"John Griffin Middle School, 5551, Fisher Road,...",North Carolina,57.29,130,0.653690
370630015,35.99167,-78.89639,"414, East Main Street, Downtown Durham, Durham...",North Carolina,122.47,181,0.666587
370630099,35.88917,-78.87444,"Research Triangle Park, Durham County, North C...",North Carolina,97.69,175,0.834965
370650099,35.98889,-77.58667,"Leggett Chapel Baptist Church, 406, NC 97, Leg...",North Carolina,17.10,77,0.654406
370670022,36.11056,-80.22667,"1227, North Dunleith Avenue, Winston-Salem, Fo...",North Carolina,278.36,291,0.696680
...,...,...,...,...,...,...,...
220150008,32.53639,-93.74917,"Shreveport Downtown Airport, Airport Drive, Sh...",Louisiana,51.89,281,0.734416
160499991,46.27556,-116.02139,"Woodland Road, Idaho County, Idaho, 83536, Uni...",Idaho,947.64,491,0.882732
120090007,28.05417,-80.62805,"3794, Town Square Boulevard, Magnolia Village,...",Florida,6.75,3,0.342489


In [6]:
airnow = pd.read_csv("/xace/d1/hamer/AirNow.csv")
airnow

,Unnamed: 0,31101,40203,40302,41302,50204,50308,50311,50604,52301,...,530090013,530330080,530570011,530730005,540390020,550030010,550270001,550790026,560030002,560450003
0,2019-07-01 13:00:00,24.0,24.0,19.0,19.0,15.0,12.0,16.0,14.0,10.0,...,22.0,22.0,4.0,2.0,15.0,23.0,20.0,19.0,26.0,19.0
1,2019-07-01 14:00:00,24.0,22.0,21.0,19.0,18.0,13.0,18.0,15.0,11.0,...,23.0,5.0,10.0,4.0,23.0,25.0,22.0,25.0,23.0,20.0
2,2019-07-01 15:00:00,23.0,19.0,21.0,20.0,18.0,15.0,19.0,18.0,14.0,...,23.0,3.0,19.0,12.0,29.0,-999000.0,30.0,29.0,36.0,20.0
3,2019-07-01 16:00:00,22.0,19.0,21.0,21.0,19.0,17.0,25.0,22.0,20.0,...,24.0,6.0,20.0,18.0,44.0,-999000.0,39.0,31.0,39.0,25.0
4,2019-07-01 17:00:00,23.0,20.0,20.0,24.0,20.0,22.0,31.0,22.0,23.0,...,23.0,7.0,19.0,23.0,56.0,28.0,39.0,42.0,41.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4398,2019-12-31 19:00:00,41.0,38.0,30.0,43.0,25.0,30.0,34.0,24.0,32.0,...,40.0,27.0,24.0,27.0,34.0,38.0,36.0,30.0,34.0,44.0
4399,2019-12-31 20:00:00,41.0,39.0,30.0,44.0,25.0,31.0,34.0,24.0,32.0,...,40.0,26.0,25.0,26.0,34.0,39.0,37.0,31.0,38.0,44.0
4400,2019-12-31 21:00:00,41.0,39.0,30.0,45.0,19.0,30.0,35.0,24.0,31.0,...,40.0,25.0,25.0,24.0,34.0,39.0,37.0,31.0,39.0,44.0
4401,2019-12-31 22:00:00,41.0,39.0,30.0,45.0,14.0,33.0,35.0,23.0,30.0,...,40.0,26.0,22.0,23.0,33.0,36.0,37.0,30.0,38.0,44.0


In [7]:
locs = locs.loc[airnow.columns[1:].astype('int64')]

In [8]:
locs = locs[locs['elevation']>=0]
X = locs[['lat','lon','elevation']].to_numpy()
X = locs[['lat','lon']].to_numpy().reshape((-1,1))
#print(X)
X = StandardScaler().fit_transform(X).reshape(locs[['lat','lon']].to_numpy().shape)
print(X.shape)
X = np.append(X,StandardScaler().fit_transform(locs[['elevation','distance']].to_numpy()), axis=1)
print(X.shape)
X=X[:,[0,1,2]]
#X[:,[2]]*=2

(342, 2)
(342, 4)


In [9]:
kmeans = KMeans(n_clusters=K, random_state=0).fit(X)
print(kmeans.labels_)
print(kmeans.cluster_centers_)

[20 13 13 13  8 20 13 13 22 22 22 22 14  6  4  7  7  7  7  7  4  7  7  7
  7  7 15  7 17  7  7  7  7  7 15  7 11  9  4  4  9  4 15  0  2 12 19 19
 12 14  1 24  2 24 24  2 24 17  4 24 24 24 24 17 24 17  2  2 17  2  7  4
  2  2 24  2  4  7 24  7 17 17  7  2  2  2  2 17  4 18 18  9 15  7  9  7
 10  2 15  2 24 24  2  2  2 24  4  2  7 24  2 17 24 24  2  2 24 17 17  2
 11  0  0  6  0 11 11  6  6  5  0  6  8  8  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3 13  4  1  1  1  1  1  1
 19 13  8  8  8  8  8  8 20 21  8 20  8  8  8  1  1 14 14  9  9  9  1 18
 18 18 18 18  6  6 18 20 20 16  8  8  8  8  8 21 10 10 10  0 11  6  0 20
  8 20 13 23 20  8 20 20 20  8  8 20  8 13 20 19 19 19 19 19 19 13 13  1
 14  1  1 14 22 22 13 21 21 21 21 13 20 13  8 21 21 20 16 20  8 16 13 13
 20 13 20 21 20  8 21 20 13 20 10  1 16 23 12 14 14 14 14 12 14 14 14 14
 14 10 10 10 10 12 12 12 12 12 12 12 12 12 12 12 12 14 12 12 12 14 14 14
 12 12 12  1 14 14 14 12 18 18  0 18 11 11 11 18  9

In [10]:
tooltip = "Click me!"
r=0
m = folium.Map(location=[locs['lat'].mean(), locs['lon'].mean()], zoom_start=4)
mc = MarkerCluster()

maxrating = locs['rating'].max()
minrating = locs['rating'].min()

for i in range(len(locs.index)):
    row = locs.iloc[i]

    #print(row['lat'], row['lon'], row['name']+"</i>")
    #mc.add_child(folium.Marker(location=[row['lat'], row['lon']],
    #            popup="<i>"+str(i)+"</i>", tooltip=row['name']))
    coord = np.power((maxrating-row['rating'])/(maxrating-minrating),0.5)
    c = ramp[int(1000*coord)-1]

    random.seed(kmeans.labels_[i])
    c = Color(hsl=(kmeans.labels_[i]*0.8/max(kmeans.labels_),random.uniform(0.5,1),0.5))
    popup = "<b>Station ID:</b> <i>"+str(locs.index[i]) \
            +"</i><br><b>Location:</b> <i>"+str(row['lat'])+", "+str(row['lon']) \
            +"</i><br><b>Elevation:</b> <i>"+str(row['elevation']) \
            +"</i><br><b>Distance to Shore:</b> <i>"+str(row['distance']) \
            +"</i><br><b>Rating:</b> <i>"+str(row['rating']) \
            +"</i>"
    folium.CircleMarker(
        [row['lat'], row['lon']],
        popup=popup, tooltip="<b>Station Name:</b> "+row['name']+"<br><b>Station Cluster:</b> "+str(kmeans.labels_[i]),
        fill=True,
        fill_opacity=1,
        radius=2,
        color=c.hex
    ).add_to(m)
    #r+=1
    #if(r>203):
    #    break
    
m.add_child(mc)
m.save('map.html')
m

In [11]:
print(list(locs.index[kmeans.labels_==1]))

[51430005, 191530030, 191531579, 200910010, 201030003, 201330003, 201730010, 270031002, 271390505, 310550019, 400270049, 401091037, 401210415, 460990008, 483670081]


In [12]:
data = []
for i in range(K):
    data.append(list(locs.index[kmeans.labels_==i]))
print(data)
print(len(data))

[[40258034, 80130014, 80310002, 80450012, 80690011, 350431001, 350610008, 490137011, 560450003], [51430005, 191530030, 191531579, 200910010, 201030003, 201330003, 201730010, 270031002, 271390505, 310550019, 400270049, 401091037, 401210415, 460990008, 483670081], [40278011, 60192009, 60250005, 60371302, 60371602, 60375005, 60470003, 60590007, 60610006, 60670002, 60670006, 60670010, 60670011, 60730001, 60731008, 60773005, 60792006, 60793001, 60831008, 60832004, 60990005, 61010003, 61130004], [120050006, 120210004, 120230002, 120330004, 120330018, 120571035, 120573002, 120590004, 120712002, 120713002, 120730012, 120814012, 120814013, 120830003, 120910002, 120972002, 121010005, 121030004, 121030018, 121056005, 121130015, 121151005, 121151006, 121152002, 121171002, 121290001], [40070010, 40132005, 40191011, 40191018, 40191032, 60290011, 60379033, 60650012, 60710001, 60798005, 160010010], [80690007], [40051008, 80410013, 80677001, 80677003, 80830006, 320312002, 320330101, 350490021], [401280